In [2]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli

In [3]:
def simulate_FG(x):
    """Simulate a field goal given a state x in the form of a list, where item 0 is the Down,
    item 1 the field position,item 2 the yards to go, and item 3 the possession (-1 or 1).
    Return an updated state based on whether or not the field goal was successful"""
    D = x[0]
    YTG = x[1]
    FP = x[2]
    POS = x[3]
    kick_length = 100 - FP + 17
    log_odds = 4.24924442 -0.08634778 * kick_length
    prob = np.exp(log_odds) / (1 + np.exp(log_odds))
    made = bernoulli.rvs(prob, 0)
    if made == 1:
        return [0, 0, 110, POS]
    else:
        return [1, 10, np.round(100 - (FP - 7), 2), POS*-1]

In [13]:
def FG_attempt(x):
    """Simulate whether a field goal gets taken given a state x in the form of a list, where
    item 0 is the Down, item 1 the field position,item 2 the yards to go, and item 3 the
    possession (-1 or 1). Return 0 for go for it and 1 for kick a field goal."""
    D = x[0]
    YTG = x[1]
    FP = x[2]
    POS = x[3]
    log_odds = -0.13841007 - 0.01402377*FP - 1.03702422*YTG + 0.0168093*(YTG*FP)
    prob = np.exp(log_odds) / (1 + np.exp(log_odds))
    attempt = bernoulli.rvs(prob, 0)
    return bool(attempt)

In [5]:
simulate_FG([4, 10, 80, 1])

[0, 0, 110, 1]

In [6]:
def simulate_play(x):
    """Simulate a play given a state x in the form of a list, where item 0 is the Down,
    item 1 the field position, item 2 the yards to go, and item 3 the possession (-1 or 1).
    Return an updated state based on a simulated yards gained"""
    gained = np.random.normal(3.5, 2) #mean, stddev
    D = x[0]
    YTG = x[1]
    FP = x[2]
    POS = x[3]
    FP = np.round(FP + gained, 2)
    YTG = np.round(YTG - gained, 2)
    
    if ((D == 4) and (x[2])) >= 65 and FG_attempt([x[0], x[1], x[2], x[3]]):
        #check if we're in a field goal state, simulate field goal
        return simulate_FG(x)
    elif FP >= 100:
        #check for a touchdown, if it's a touchdown return the touchdown state:
        return [0, 0, 100, POS]
    elif YTG <= 0:
        #check for a first down, if it's a first down reset the down and yards to go
        D = 1
        YTG = 10
    elif D == 4:
        #turnover on downs, so change possession and switch the field position
        POS *= -1
        FP = np.round(100 - FP, 2)
        YTG = 10
        D = 1
    else:
        D += 1
        
    return [D, YTG, FP, POS]
    

In [7]:
simulate_play([4, 10, 80, 1])

[1, 10, 27, -1]

In [8]:
def simulate_football(D, YTG, FP, POS):
    state = [D, YTG, FP, POS]
    state = simulate_play(state)
    #while state is not a touchdown, keep running plays but once in the touchdown
    #state return 7 or -7
    while state[0] != 0:
        state = simulate_play(state) 
    
    if state[2] == 100:
        return state[3] * 7
    elif state[2] == 110:
        return state[3] * 3
    

In [9]:
simulate_football(1, 10, 25, 1)

7

In [10]:
def expected_points(state, reps):
    scores = []
    D = state[0]
    YTG = state[1]
    FP = state[2]
    POS = state[3]
    for i in range(reps):
        score = simulate_football(D, YTG, FP, POS)
        scores.append(score)

    return np.mean(scores)

In [11]:
expected_points([1, 10, 25, 1], 10000)

0.6338